In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import igraph as ig

# Importing Data and Modules 

Here, we're importing our datasets of Instagram posts - both of them :)

In [2]:
PATH_DATA = "../data/"

# Here, we're incorporating both datasets
accounts_data = pd.read_csv(PATH_DATA + "instagram_accounts.csv", converters={'id_followers': literal_eval})
posts_data1 = pd.read_csv(PATH_DATA + "instagram_posts.csv")
posts_data2 = pd.read_csv(PATH_DATA + "instagram_posts_1211_1611.csv")
posts_data = pd.concat([posts_data1, posts_data2],axis=0)


In [3]:
# Checking if our duplicate user changes anything ...
posts_data = posts_data[posts_data['id_user'] != 603282]
posts_data.reset_index(drop=True, inplace=True)

In [4]:
# Processing posts_data
# posts_data.drop(inplace=True, columns=["Unnamed: 0"])
print(posts_data.head)
posts_data['id_user'].unique().size

<bound method NDFrame.head of       Unnamed: 0  id_user    id_post        date   time half_day  views  \
0              0   672702  638779430  09/11/2017  07:53       am      2   
1              1   474227  953043456  09/11/2017  02:03       am     36   
2              2   587566  650889385  09/11/2017  02:57       am     59   
3              3   483543  144265898  09/11/2017  11:43       am     21   
4              4   394103  955542283  09/11/2017  04:53       pm     16   
...          ...      ...        ...         ...    ...      ...    ...   
3040        3042   746387  121744758  16/11/2017  00:57       am    128   
3041        3043   404047  794339349  16/11/2017  08:49       am    129   
3042        3044   630873  628294384  16/11/2017  09:59       am     47   
3043        3045   719594  358445633  16/11/2017  02:46       am    241   
3044        3046   639300  225756005  16/11/2017  06:54       am      7   

      reposts  likes  comments  id_post_origin  link_clicks  donation

3045

## _Posts_ Graph Construction

Here we construct the graph for the posts. Each node is identified by its index in the dataframe, and then, its attributes are given as node attributes. All edges are added based on this. 
- Note: this has to be a directed graph to respect  the flow of information i.e. A -> B means that information flows from A to B. 

In [5]:
# Checking for duplicates again - should be empty 
posts_data[posts_data.duplicated(subset=["id_post"])]

,Unnamed: 0,id_user,id_post,date,time,half_day,views,reposts,likes,comments,id_post_origin,link_clicks,donation_tag,donation_val,house_buy


In [6]:
## Construct the graph, and add vertices
posts = ig.Graph(directed=True); 
posts.add_vertices(posts_data.index.values)

In [7]:
for idx in posts_data.index: 
    ### if id_post_origin is zero, append it to its own list
    #   These are the original posts.
    id_origin, id_post, id_user = posts_data.iloc[idx][['id_post_origin', 'id_post','id_user']] 
    # Add the vertex properties first 
    posts.vs[idx]['id_post'] = id_post
    posts.vs[idx]['id_post_origin'] = id_origin
    posts.vs[idx]['id_user'] = id_user    
    
    # Now, if it's not an origin post, 
    # we can add the corresponding edge. 
    if id_origin != 0 :
        # For this, we take the id_origin and 
        # get the index of that post. Then, 
        # we can connect those two nodes :)
        orig_index = posts_data.index[posts_data['id_post'] == id_origin].values[0]
        posts.add_edges([ (orig_index, idx) ])

posts.vs['size']=7
posts.vs['arrow_size']=1
posts.vs['arrow_width']=1
ig.plot(posts)

## Components 

We expect our data to have 4 weakly connected components, since we're working with a DAG, and because there are 4 'original' posts which have been reposted over and over again.

In [8]:
# A routine check: 
print("Multiple edges: ", posts.has_multiple())
components = posts.clusters(mode='weak')
print(components)

Multiple edges:  False
Clustering with 3045 elements and 4 clusters
[0] 0, 4, 7, 9, 10, 13, 16, 22, 26, 27, 29, 30, 32, 36, 39, 41, 42, 43, 54,
    58, 59, 61, 66, 69, 70, 75, 77, 82, 83, 84, 85, 87, 91, 92, 94, 97, 98,
    100, 101, 104, 106, 107, 109, 110, 111, 113, 122, 124, 128, 134, 135, 137,
    140, 141, 142, 149, 161, 163, 164, 165, 166, 171, 174, 175, 178, 179, 183,
    184, 186, 187, 188, 194, 196, 197, 198, 202, 203, 204, 205, 214, 220, 222,
    224, 225, 227, 229, 231, 234, 236, 237, 241, 242, 246, 247, 250, 251, 252,
    253, 261, 268, 271, 273, 276, 279, 280, 281, 283, 287, 290, 293, 294, 295,
    301, 303, 304, 307, 312, 316, 317, 318, 322, 324, 326, 327, 329, 332, 333,
    334, 336, 338, 340, 341, 343, 349, 353, 366, 368, 371, 372, 376, 378, 388,
    389, 393, 399, 401, 404, 406, 407, 410, 413, 414, 417, 418, 419, 420, 421,
    422, 427, 428, 430, 432, 436, 437, 438, 439, 442, 443, 445, 446, 447, 449,
    450, 454, 456, 461, 465, 466, 467, 471, 483, 485, 490, 491, 496, 

# Metric 1: Interactivity

Here, we implement the first metric proposed in the first deliverable. This measures the number of likes, clicks, reposts, donations etc. that each post has. We can use this to create a composite ranking, which then provides us a KPI to maximise. 

In [9]:
# We get the nodes representing the original posts 
# i.e. have original post ID = 0.
original_posts = posts.vs(id_post_origin_eq=0)
for post in original_posts:
    print(post)

igraph.Vertex(<igraph.Graph object at 0x0000016221D89130>, 0, {'name': 0, 'id_post': 638779430, 'id_post_origin': 0, 'id_user': 672702, 'size': 7, 'arrow_size': 1, 'arrow_width': 1})
igraph.Vertex(<igraph.Graph object at 0x0000016221D89130>, 1, {'name': 1, 'id_post': 953043456, 'id_post_origin': 0, 'id_user': 474227, 'size': 7, 'arrow_size': 1, 'arrow_width': 1})
igraph.Vertex(<igraph.Graph object at 0x0000016221D89130>, 2, {'name': 2, 'id_post': 650889385, 'id_post_origin': 0, 'id_user': 587566, 'size': 7, 'arrow_size': 1, 'arrow_width': 1})
igraph.Vertex(<igraph.Graph object at 0x0000016221D89130>, 3, {'name': 3, 'id_post': 144265898, 'id_post_origin': 0, 'id_user': 483543, 'size': 7, 'arrow_size': 1, 'arrow_width': 1})
